The examples below are exercises from the code here: https://github.com/google/or-tools/tree/master/examples/python

# puzzle 1

 A merchant had a forty pound measuring weight that broke
 into four pieces as the result of a fall. When the pieces were
 subsequently weighed, it was found that the weight of each piece
 was a whole number of pounds and that the four pieces could be
 used to weigh every integral weight between 1 and 40 pounds. What
 were the weights of the pieces?
 
 Note that since this was a 17th-century merchant, he of course used a
 balance scale to weigh things. So, for example, he could use a 1-pound
 weight and a 4-pound weight to weigh a 3-pound object, by placing the
 3-pound object and 1-pound weight on one side of the scale, and
 the 4-pound weight on the other side.

In [ ]:
from ortools.constraint_solver import pywrapcp


solver = pywrapcp.Solver('Broken weights')
n_weights = 4
W = 40

weights = [solver.IntVar(1, W) for _ in range(n_weights)]

x = {}
for target_weight in range(1,W+1):
    for wgt in range(n_weights):
        x[(target_weight, wgt)] = solver.IntVar(-1, 1)
x_flat = [x[(target_weight, wgt)] for target_weight in range(1,W+1) for wgt in range(n_weights)]
        
# symmetry breaking
for j in range(1, n_weights):
    solver.Add(weights[j - 1] < weights[j])

# weights must sum to W
solver.Add(solver.SumEquality(weights, W))

# Check that all weights from 1 to 40 can be made.
#
# Since all weights can be on either side
# of the side of the scale we allow either
# -1, 0, or 1 or the weights, assuming that
# -1 is the weights on the left and 1 is on the right.
#
for target_weight in range(1, W+1):
    solver.Add(solver.Sum([x[(target_weight, wgt)] * weights[wgt] for wgt in range(n_weights)]) == target_weight)
    

db = solver.Phase(weights + x_flat,
                  solver.CHOOSE_FIRST_UNBOUND,
                  solver.ASSIGN_MIN_VALUE)

solver.Solve(db)


In [ ]:
objective = solver.Minimize(weights[n_weights - 1], 1)

search_log = solver.SearchLog(1)

solver.NewSearch(db, [objective])
num_solutions = 0
while solver.NextSolution():
    num_solutions += 1
    print('weights:   ', end=' ')
    for w in [weights[j].Value() for j in range(n_weights)]:
        print('%3i ' % w, end=' ')
    print()
    print('-' * 30)
    for i in range(1,W+1):
        print('weight  %2i:' % (i), end=' ')
        for j in range(n_weights):
            print('%3i ' % x[i, j].Value(), end=' ')
            print()
        print()
    print()
solver.EndSearch()

print('num_solutions:', num_solutions)
print('failures :', solver.Failures())
print('branches :', solver.Branches())
print('WallTime:', solver.WallTime(), 'ms')

## puzzle 2

  '''
  What is the minimum number of coins that allows one to pay _exactly_
  any amount smaller than one Euro? Recall that there are six different
  euro cents, of denomination 1, 2, 5, 10, 20, 50
  '''

In [ ]:
from ortools.constraint_solver import pywrapcp
solver = pywrapcp.Solver('euros')

base_values = [1, 2, 5, 10, 25, 50]
n_bases = len(base_values)


x = [solver.IntVar(0, 99, 'coin_{coin}'.format(coin=coin)) for coin in base_values]
num_coins = solver.IntVar(0, 20, "num_coins")
solver.AddConstraint(num_coins == solver.Sum(x))
#num_coins == solver.Sum(x)


for target_value in range(1, 100):
    aloc = [solver.IntVar(0,20, 'aloc_{coin}'.format(coin=coin)) for coin in base_values]
    solver.AddConstraint(solver.ScalProd(aloc, base_values) == target_value)
    [solver.AddConstraint(aloc[base] <= x[base]) for base in range(n_bases)] 


In [ ]:
objective = solver.Minimize(num_coins, 1)

#solution and search
solution = solver.Assignment()
solution.Add(x)
solution.Add(num_coins)
solution.AddObjective(num_coins)

db = solver.Phase(x,
                  solver.CHOOSE_MIN_SIZE_LOWEST_MAX,
                  solver.ASSIGN_MIN_VALUE)

solver.NewSearch(db, [objective])
num_solutions = 0
while solver.NextSolution():
    print("x: ", [x[i].Value() for i in range(n_bases)])
    print("num_coins:", num_coins.Value())
    print()
    num_solutions += 1
solver.EndSearch()

print()
print("num_solutions:", num_solutions)
print("failures:", solver.Failures())
print("branches:", solver.Branches())
print("WallTime:", solver.WallTime())



## puzzle 3

  '''
  In a quadratic grid (or a larger chessboard) with 31x31 cells, one should
  place coins in such a way that the following conditions are fulfilled:
     1. In each row exactly 14 coins must be placed.
     2. In each column exactly 14 coins must be placed.
     3. The sum of the quadratic horizontal distance from the main diagonal
        of all cells containing a coin must be as small as possible.
     4. In each cell at most one coin can be placed.
  The description says to place 14x31 = 434 coins on the chessboard each row
  containing 14 coins and each column also containing 14 coins.
  '''

In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('CoinsGridCBC',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

X = {}
board_size = 31
for row in range(board_size):
    for col in range(board_size):
        X[(row, col)] = solver.IntVar(0, 1, 'X_({row},{col})'.format(row=row, col=col))
        
X_flat = [X[(row, col)] for row in range(board_size) for col in range(board_size)]

# the sum to 14 constraints
for row in range(board_size):
    solver.Add(solver.Sum([X[row, col] for col in range(board_size)])==14)

for col in range(board_size):
    solver.Add(solver.Sum([X[row, col] for row in range(board_size)])==14) 
    
### the thing to miminize
obj = solver.Sum([(col-row)*(col-row) for col in range(board_size) for row in range(board_size) if X[(row, col)]==1])

objective = solver.Minimize(obj)

solver.Solve()

for row in range(board_size):
    for col in range(board_size):
        # int representation
        print(int(X[(row, col)].SolutionValue())),
    print()

print('walltime  :', solver.WallTime(), 'ms')


## puzzle 4


  This example is from the OPL example covering.mod
  '''
  Consider selecting workers to build a house. The construction of a
  house can be divided into a number of tasks, each requiring a number of
  skills (e.g., plumbing or masonry). A worker may or may not perform a
  task, depending on skills. In addition, each worker can be hired for a
  cost that also depends on his qualifications. The problem consists of
  selecting a set of workers to perform all the tasks, while minimizing the
  cost. This is known as a set-covering problem. The key idea in modeling
  a set-covering problem as an integer program is to associate a 0/1
  variable with each worker to represent whether the worker is hired.
  To make sure that all the tasks are performed, it is sufficient to
  choose at least one worker by task. This constraint can be expressed by a
  simple linear inequality.
  '''

In [ ]:
### GIVEN DATA ####
n_workers = 32
Workers = list(range(n_workers))
n_tasks = 15
Tasks = list(range(n_tasks))

# Which worker is qualified for each task.
qualified_workers_per_task = [
    [1, 9, 19, 22, 25, 28, 31],
    [2, 12, 15, 19, 21, 23, 27, 29, 30, 31, 32],
    [3, 10, 19, 24, 26, 30, 32],
    [4, 21, 25, 28, 32],
    [5, 11, 16, 22, 23, 27, 31],
    [6, 20, 24, 26, 30, 32],
    [7, 12, 17, 25, 30, 31],
    [8, 17, 20, 22, 23],
    [9, 13, 14, 26, 29, 30, 31],
    [10, 21, 25, 31, 32],
    [14, 15, 18, 23, 24, 27, 30, 32],
    [18, 19, 22, 24, 26, 29, 31],
    [11, 20, 25, 28, 30, 32],
    [16, 19, 23, 31],
    [9, 18, 26, 28, 31, 32]]


cost_per_worker = [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 
        3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 8, 9]

In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('LinearExample',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x = [solver.IntVar(0, 1, 'hire_worker_{wrk}'.format(wrk=wrk)) for wrk in Workers]


## make an array [n_workers][n_tasks] with 1 or 0 based on weather worker can do task
competency_matrix = []
for tsk in Tasks:
    worker_list = []
    for wrk in Workers:
        if (wrk+1) in qualified_workers_per_task[tsk]:
            worker_list.append(1)
        else:
            worker_list.append(0)
    competency_matrix.append(worker_list)

## At least one worker per task
for tsk in Tasks:
    solver.Add(solver.Sum([competency_matrix[tsk][wrk]*x[wrk] for wrk in Workers])>=1)

# mimize cost
objective_value = solver.Sum([cost_per_worker[wrk]*x[wrk] for wrk in Workers])
objective = solver.Minimize(objective_value)

status = solver.Solve()



In [ ]:
for wrk in Workers:
    print(wrk, int(x[wrk].SolutionValue()))

## puzzle 5

simple diet problem using MIP in Google CP Solver.
  Standard Operations Research example.
  Minimize the cost for the products:
  Type of                        Calories   Chocolate    Sugar    Fat
  Food                                      (ounces)     (ounces) (ounces)
  Chocolate Cake (1 slice)       400           3            2      2
  Chocolate ice cream (1 scoop)  200           2            2      4
  Cola (1 bottle)                150           0            4      1
  Pineapple cheesecake (1 piece) 500           0            4      5


In [ ]:
### GIVEN DATA ###

import sys
from ortools.linear_solver import pywraplp


cost = [50, 20, 30, 80]  # in cents
requirements = [500, 6, 10, 8]  # requirements for each nutrition type

# nutritions for each product
calories = [400, 200, 150, 500]
chocolate = [3, 2, 0, 0]
sugar = [2, 2, 4, 4]
fat = [2, 4, 1, 5]

n_reqs = len(requirements)
n_prods = len(cost)

prod_names = ['cake', 'cream', 'cola', 'cheese']

nutrition_matrix = [calories, chocolate, sugar, fat] #[nut][prod]

solver = pywraplp.Solver('LinearExample',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)



x = [solver.IntVar(0, 99, '{name}'.format(name=prod)) for prod in prod_names]

# must achieve nutrution targets
for nut in range(n_reqs):
    solver.Add(solver.Sum([x[prod]*nutrition_matrix[nut][prod]
                           for prod in range(n_prods)]) >= requirements[nut])
    
# minimize cost: 
cost = solver.Sum([x[prod]*cost[prod] for prod in range(n_prods)])
solver.Minimize(cost)
solver.Solve()
print('total price', solver.Objective().Value())
for i in range(n_prods):
    print(x[i], x[i].SolutionValue())

print('time:',  solver.WallTime())
    
    

## puzzle 6


  This is a little 'tomography' problem, taken from an old issue
  of Scientific American.
  A matrix which contains zeroes and ones gets "x-rayed" vertically and
  horizontally, giving the total number of ones in each row and column.
  The problem is to reconstruct the contents of the matrix from this
  information. https://github.com/google/or-tools/blob/master/examples/python/discrete_tomography.py

In [ ]:
### GIVEN DATA ###
row_sums = [0, 0, 8, 2, 6, 4, 5, 3, 7, 0, 0]
col_sums = [0, 0, 7, 1, 6, 3, 4, 5, 2, 7, 0, 0]
#####

n_rows = len(row_sums)
n_cols = len(col_sums)

from ortools.constraint_solver import pywrapcp
solver = pywrapcp.Solver("n-queens")

# create vars
X = {}
for row in range(n_rows):
    for col in range(n_cols):
        X[(row, col)] = solver.IntVar(0, 1, 'x_{row}_{col}'.format(row=row, col=col))


# row constraints:
for row in range(n_rows):
    solver.Add(solver.Sum([X[(row, col)] for col in range(n_cols)]) == row_sums[row])

for col in range(n_cols):
    solver.Add(solver.Sum([X[(row, col)] for row in range(n_rows)]) == col_sums[col])

    
x = [X[(row, col)] for row in range(n_rows) for col in range(n_cols)]


solution = solver.Assignment()
solution.Add(x)

db = solver.Phase(x,
                  solver.INT_VAR_SIMPLE,
                  solver.ASSIGN_MIN_VALUE)

solver.NewSearch(db)
num_solutions = 0
solution_mat = []
while solver.NextSolution():
    print('hey!')
    for row in range(n_rows):
        row_list = [0]*n_cols
        for col in range(n_cols):
            if X[(row, col)].Value()==1:
                row_list[col]=1
        solution_mat.append(row_list)
    num_solutions += 1
solver.EndSearch()

print()
print("num_solutions:", num_solutions)
print("failures:", solver.Failures())
print("branches:", solver.Branches())
print("WallTime:", solver.WallTime())
    
print()
for i in solution_mat:
    print(i)



## puzzle 7

"""Gate Scheduling problem.
We have a set of jobs to perform (duration, width).
We have two parallel machines that can perform this job.
One machine can only perform one job at a time.
At any point in time, the sum of the width of the two active jobs does not
exceed a max_length.
The objective is to minimize the max end time of all jobs.

https://github.com/google/or-tools/blob/master/examples/python/gate_scheduling_sat.py

In [ ]:
# GIVEN DATA # 

# (duration, width)
job_specs = [[3, 3],
        [2, 5],
        [1, 3],
        [3, 7],
        [7, 3],
        [2, 2],
        [2, 2],
        [5, 5],
        [10, 2],
        [4, 3],
        [2, 6],
        [1, 2],
        [6, 8],
        [4, 5],
        [3, 7]]

max_length = 10


# Conjunctive constraints: if certain tasks must be done in certain orders
# Disjunctive constraints: a task executioner can only work on one task at a time.

from ortools.constraint_solver import pywrapcp
solver = pywrapcp.Solver('jobshop')

## Create the allocation variable
n_jobs = len(job_specs)
n_machines = 2
job_list = range(n_jobs)
machine_list = range(n_machines)

horizon = 0
for job in job_specs:
    horizon += job[0]
print('horizon is', horizon)

try:
    job_intervals = [solver.FixedDurationIntervalVar(0,horizon,
                                                     job_specs[job][0],
                                                     False,
                                                     'job_{job}_int'.format(job=job))
                     for job in job_list]

    # a job gets assigned one and only one machine
    job_machine_alloc = [solver.IntVar(0, n_machines-1, 'job_{job}_mch'.format(job=job)) for job in job_list]

    
    #
    # disjunctive constraints (if jobs are in the same machine, they can't overlap)
    # ... don't know how to do conditional disjunctive
    
    ## the width constrinat
    # solver.AddCumulative(job_intervals, [job[1] for job in job_specs], max_length)

    ## objective value
    # obj_value = solver.Max([job_intervals[job].EndExpr()
    #                        for job in job_list])

    # MINIMIZE # 
    # objective_monitor = solver.Minimize(obj_value, 1)

    ##
    # db = solver.Phase([obj_value],
    #                  solver.CHOOSE_FIRST_UNBOUND,
    #                  solver.ASSIGN_MIN_VALUE)

    # solver.Solver(db)
except:
    print('asd')

In [ ]:
from ortools.sat.python import cp_model
from ortools.sat.python import visualization


def main():
  model = cp_model.CpModel()

  jobs = [[3, 3],
          [2, 5],
          [1, 3],
          [3, 7],
          [7, 3],
          [2, 2],
          [2, 2],
          [5, 5],
          [10, 2],
          [4, 3],
          [2, 6],
          [1, 2],
          [6, 8],
          [4, 5],
          [3, 7]]

  max_length = 10

  horizon = sum(t[0] for t in jobs)
  num_jobs = len(jobs)
  all_jobs = range(num_jobs)

  intervals = []
  intervals0 = []
  intervals1 = []
  performed = []
  starts = []
  ends = []
  demands = []

  for i in all_jobs:
    # Create main interval.
    start = model.NewIntVar(0, horizon, 'start_%i' % i)
    duration = jobs[i][0]
    end = model.NewIntVar(0, horizon, 'end_%i' % i)
    interval = model.NewIntervalVar(start, duration, end, 'interval_%i' % i)
    starts.append(start)
    intervals.append(interval)
    ends.append(end)
    demands.append(jobs[i][1])

    performed_on_m0 = model.NewBoolVar('perform_%i_on_m0' % i)
    performed.append(performed_on_m0)

    # Create an optional copy of interval to be executed on machine 0.
    start0 = model.NewOptionalIntVar(
        0, horizon, performed_on_m0, 'start_%i_on_m0' % i)
    end0 = model.NewOptionalIntVar(
        0, horizon, performed_on_m0, 'end_%i_on_m0' % i)
    interval0 = model.NewOptionalIntervalVar(
        start0, duration, end0, performed_on_m0, 'interval_%i_on_m0' % i)
    intervals0.append(interval0)

    # Create an optional copy of interval to be executed on machine 1.
    start1 = model.NewOptionalIntVar(
        0, horizon, performed_on_m0.Not(), 'start_%i_on_m1' % i)
    end1 = model.NewOptionalIntVar(
        0, horizon, performed_on_m0.Not(), 'end_%i_on_m1' % i)
    interval1 = model.NewOptionalIntervalVar(
        start1, duration, end1, performed_on_m0.Not(), 'interval_%i_on_m1' % i)
    intervals1.append(interval1)

    # We only propagate the constraint if the tasks is performed on the machine.
    model.Add(start0 == start).OnlyEnforceIf(performed_on_m0)
    model.Add(start1 == start).OnlyEnforceIf(performed_on_m0.Not())

  # Max Length constraint (modeled as a cumulative)
  model.AddCumulative(intervals, demands, max_length)

  # Choose which machine to perform the jobs on.
  model.AddNoOverlap(intervals0)
  model.AddNoOverlap(intervals1)

  # Objective variable.
  makespan = model.NewIntVar(0, horizon, 'makespan')
  model.AddMaxEquality(makespan, ends)
  model.Minimize(makespan)

  # Symmetry breaking.
  model.Add(performed[0] == 0)

  # Solve model.
  solver = cp_model.CpSolver()
  solver.Solve(model)


  # Output solution.
  if visualization.RunFromIPython():
    output = visualization.SvgWrapper(solver.ObjectiveValue(), max_length, 40.0)
    output.AddTitle('Makespan = %i' % solver.ObjectiveValue())
    color_manager = visualization.ColorManager()
    color_manager.SeedRandomColor(0)

    for i in all_jobs:
      performed_machine = 1 - solver.Value(performed[i])
      start = solver.Value(starts[i])
      dx = jobs[i][0]
      dy = jobs[i][1]
      sy = performed_machine * (max_length - dy)
      output.AddRectangle(start, sy, dx, dy, color_manager.RandomColor(),
                          'black', 'j%i' % i)

    output.AddXScale()
    output.AddYScale()
    output.Display()
  else:
    print('Solution')
    print('  - makespan = %i' % solver.ObjectiveValue())
    for i in all_jobs:
      performed_machine = 1 - solver.Value(performed[i])
      start = solver.Value(starts[i])
      print('  - Job %i starts at %i on machine %i' %
            (i, start, performed_machine))
    print('Statistics')
    print('  - conflicts : %i' % solver.NumConflicts())
    print('  - branches  : %i' % solver.NumBranches())
    print('  - wall time : %f ms' % solver.WallTime())


if __name__ == '__main__':
  main()

## puzzle 8

  '''
  A kid goes into a grocery store and buys four items. The cashier
  charges 7.11, the kid pays and is about to leave when the cashier
  calls the kid back, and says 'Hold on, I multiplied the four items
  instead of adding them; I'll try again; Hah, with adding them the
  price still comes to 7.11'. What were the prices of the four items?
  '''
  
  https://github.com/google/or-tools/blob/master/examples/python/grocery.py

In [ ]:
from ortools.constraint_solver import pywrapcp
from functools import reduce

solver = pywrapcp.Solver('four items')

total_price = 711
n_items = 4
items = range(n_items)

prices = [solver.IntVar(0, total_price, 'item_{itm}'.format(itm=itm)) for itm in items]

# total price constraint
solver.AddConstraint(solver.SumEquality(prices, total_price))

# multiplicative price constraint
solver.Add(reduce(lambda x, y: x * y, prices) == total_price * 100 ** 3)

## break symetry of items
for itm in range(1, n_items):
    solver.Add(prices[itm - 1] < prices[itm])


solution = solver.Assignment()
solution.Add(prices)

db = solver.Phase(prices,
                  solver.INT_VAR_SIMPLE,
                  solver.ASSIGN_MIN_VALUE)

solver.NewSearch(db)
while solver.NextSolution():
    print('hey!')
    for p in prices:
        print(p, p.Value())
solver.EndSearch()


## puzzle 9 - jobshop

"""This model implements a simple jobshop named ft06.
A jobshop is a standard scheduling problem when you must sequence a
series of tasks on a set of machines. Each job contains one task per
machine. The order of execution and the length of each job on each
machine is task dependent.
The objective is to minimize the maximum completion time of all
jobs. This is called the makespan.
"""

In [ ]:
from ortools.constraint_solver import pywrapcp
solver = pywrapcp.Solver('job6')

# duration of each task in each job. durations[job][task]->duration
durations = [[1, 3, 6, 7, 3, 6],
             [8, 5, 10, 10, 10, 4],
             [5, 4, 8, 9, 1, 7],
             [5, 5, 5, 3, 8, 9],
             [9, 3, 5, 4, 3, 1],
             [3, 3, 9, 10, 4, 1]]

# machines that each task needs to be exectuted in. machines[job][task]->machine
machines = [[2, 0, 1, 3, 5, 4],
              [1, 2, 4, 5, 0, 3],
              [2, 3, 5, 0, 1, 4],
              [1, 0, 2, 3, 4, 5],
              [2, 1, 4, 5, 0, 3],
              [1, 3, 5, 0, 4, 2]]


n_jobs = len(durations)
n_tasks = len(durations[0])
n_machines = 6
jobs_list = range(n_jobs)
tasks_list = range(n_tasks)
machine_list = range(n_machines)


horizon = sum([sum(durations[i]) for i in jobs_list])


all_tasks = {}
for job in jobs_list:
    for tsk in tasks_list:
        all_tasks[(job, tsk)] = solver.FixedDurationIntervalVar(0, horizon,
                                                                durations[job][tsk],
                                                                False, 
                                                                'job_{job}_task_{tsk}  '.format(job=job, tsk=tsk)
                                                               )

# Conjunctive constraints: if certain tasks must be done in certain orders
# Disjunctive constraints: a task executioner can only work on one task at a time.

#### DISJUNCTIVE
all_sequences = []
for mch in machine_list:
    tasks_for_machine = []
    for job in jobs_list:
        for tsk in tasks_list:
            if machines[job][tsk] == mch:
                tasks_for_machine.append(all_tasks[(job, tsk)])
    disj = solver.DisjunctiveConstraint(tasks_for_machine, 'dis_mch{mch}'.format(mch=mch))
    all_sequences.append(disj.SequenceVar())
    solver.Add(disj)

obj_value = solver.Max([all_tasks[(job, tsk)].EndExpr() 
                        for job in jobs_list for tsk in tasks_list])
minimizer = solver.Minimize(obj_value, 1)


collector = solver.LastSolutionCollector()
collector.AddObjective(obj_value)
collector.Add(all_sequences)

for mch in machine_list:
    sequence = all_sequences[mch];
    sequence_count = sequence.Size();
    for j in range(0, sequence_count):
        t = sequence.Interval(j)
        collector.Add(t.StartExpr().Var())
        collector.Add(t.EndExpr().Var())


# Create search phases.
sequence_phase = solver.Phase([all_sequences[mch] for mch in machine_list],
                              solver.SEQUENCE_DEFAULT)

vars_phase = solver.Phase([obj_value],
                          solver.CHOOSE_FIRST_UNBOUND,
                          solver.ASSIGN_MIN_VALUE)

main_phase = solver.Compose([sequence_phase, vars_phase])


solver.Solve(main_phase, [minimizer, collector])

print('makespan:', collector.ObjectiveValue(0))
print('bt')

# stuff for visuals
sol_line = ""
sol_line_tasks = ""
disp_col_width=10

for mch in machine_list:
    seq = all_sequences[mch]
    sol_line += "Machine " + str(mch) + ": "
    sol_line_tasks += "Machine " + str(mch) + ": "
    sequence = collector.ForwardSequence(0, seq)
    seq_size = len(sequence)


    for j in range(0, seq_size):
        t = seq.Interval(sequence[j]);
        # Add spaces to output to align columns.
        sol_line_tasks +=  t.Name() + "   " * (disp_col_width - len(t.Name()))

    for j in range(0, seq_size):
        t = seq.Interval(sequence[j]);
        sol_tmp = "[" + str(collector.Value(0, t.StartExpr().Var())) + ","
        sol_tmp += str(collector.Value(0, t.EndExpr().Var())) + "] "
        # Add spaces to output to align columns.
        sol_line += sol_tmp + " " * (disp_col_width - len(sol_tmp))
    
    print(sol_line_tasks)
    print("Time Intervals for Tasks\n")
    print(sol_line)
#machine0: (0,1) (1,4) (2,3) (3,1) (4,4) (5,3)

In [ ]:
int0 = all_sequences[3]

In [ ]:
int0.Interval(0)

## problem 10 - knapsack

https://github.com/google/or-tools/blob/master/examples/python/knapsack.py

In [ ]:
from ortools.algorithms import pywrapknapsack_solver

solver = pywrapknapsack_solver.KnapsackSolver(
      pywrapknapsack_solver.KnapsackSolver.
      KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
      'test')
profits = [360, 83, 59, 130, 431, 67, 230, 52, 93,
             125, 670, 892, 600, 38, 48, 147, 78, 256,
             63, 17, 120, 164, 432, 35, 92, 110, 22,
             42, 50, 323, 514, 28, 87, 73, 78, 15,
             26, 78, 210, 36, 85, 189, 274, 43, 33,
             10, 19, 389, 276, 312]
costs = [[7, 0, 30, 22, 80, 94, 11, 81, 70,
              64, 59, 18, 0, 36, 3, 8, 15, 42,
              9, 0, 42, 47, 52, 32, 26, 48, 55,
              6, 29, 84, 2, 4, 18, 56, 7, 29,
              93, 44, 71, 3, 86, 66, 31, 65, 0,
              79, 20, 65, 52, 13]]

capacities = [850]

solver.Init(profits, costs, capacities)

solution = solver.Solve()

packed_items = [x for x in range(0, len(weights[0]))
                    if solver.BestSolutionContains(x)]

packed_weights = [weights[0][i] for i in packed_items]

print("Packed items: ", packed_items)
print("Packed weights: ", packed_weights)
print("Total weight (same as total value): ", solution)



In [ ]:
# Multidimensional variant
from ortools.algorithms import pywrapknapsack_solver

solver = pywrapknapsack_solver.KnapsackSolver(
      pywrapknapsack_solver.KnapsackSolver.
      KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER,
      'test')

capacities = [1820, 769, 133, 92, 2663, 6118, 1336]
profits = [96, 76, 56, 11, 86, 10, 66, 86, 83, 12, 9, 81]
costs = [[19, 1, 10, 1, 1, 14, 152, 11, 1, 1, 1, 1],
         [0, 4, 53, 0, 0, 80, 0, 4, 5, 0, 0, 0],
         [4, 660, 3, 0, 30, 0, 3, 0, 4, 90, 0, 0],
         [7, 0, 18, 6, 770, 330, 7, 0, 0, 6, 0, 0],
         [0, 20, 0, 4, 52, 3, 0, 0, 0, 5, 4, 0],
         [0, 0, 40, 70, 4, 63, 0, 0, 60, 0, 4, 0],
         [0, 32, 0, 0, 0, 5, 0, 3, 0, 660, 0, 9]]

solver.Init(profits, costs, capacities)

solution = solver.Solve()
n_items = len(profits)
packed_items = [x for x in range(n_items)
                    if solver.BestSolutionContains(x)]

print("Packed items: ", packed_items)
print("Total weight (same as total value): ", solution)

In [ ]:
# the example above as a MLIP
from ortools.linear_solver import pywraplp
from functools import reduce

solver = pywraplp.Solver('multim-knapsack',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


capacities = [1820, 769, 133, 92, 2663, 6118, 1336]
profits = [96, 76, 56, 11, 86, 10, 66, 86, 83, 12, 9, 81]
costs = [[19, 1, 10, 1, 1, 14, 152, 11, 1, 1, 1, 1],
         [0, 4, 53, 0, 0, 80, 0, 4, 5, 0, 0, 0],
         [4, 660, 3, 0, 30, 0, 3, 0, 4, 90, 0, 0],
         [7, 0, 18, 6, 770, 330, 7, 0, 0, 6, 0, 0],
         [0, 20, 0, 4, 52, 3, 0, 0, 0, 5, 4, 0],
         [0, 0, 40, 70, 4, 63, 0, 0, 60, 0, 4, 0],
         [0, 32, 0, 0, 0, 5, 0, 3, 0, 660, 0, 9]]


n_dim = len(capacities)
n_items = len(profits)


x = [solver.IntVar(0,1, 'x_{itm}'.format(itm=itm)) for itm in range(n_items)]


# constraints
for cap in range(n_dim):
    solver.Add(solver.Sum([x[itm]*costs[cap][itm]
                           for itm in range(n_items)]) <= capacities[cap])
    
# maximize
obj_value = solver.Sum([x[itm]*profits[itm]
                        for itm in range(n_items)])
objective = solver.Maximize(obj_value)

solver.Solve()

print()
print('obj_value: ', int(solver.Objective().Value()))

print('take:', end=' ')
for i in range(n_items):
    print(int(x[i].SolutionValue()), end=' ')


## puzzle 11

https://github.com/google/or-tools/blob/master/examples/python/labeled_dice.py
    
  My daughter Jenn bough a puzzle book, and showed me a cute puzzle.  There
  are 13 words as follows:  BUOY, CAVE, CELT, FLUB, FORK, HEMP, JUDY,
  JUNK, LIMN, QUIP, SWAG, VISA, WISH.
  There are 24 different letters that appear in the 13 words.  The question
  is:  can one assign the 24 letters to 4 different cubes so that the
  four letters of each word appears on different cubes.  (There is one
  letter from each word on each cube.)  It might be fun for you to try
  it.  I'll give a small hint at the end of this post. The puzzle was
  created by Humphrey Dudley.

In [ ]:
import string
alphabet = string.ascii_uppercase
num_to_let_dict={k: v for k, v in enumerate(alphabet)}
let_to_num_dict={v: k for k, v in enumerate(alphabet)}


words = ['BUOY', 'CAVE', 'CELT', 'FLUB', 'FORK', 
         'HEMP', 'JUDY', 'JUNK', 'LIMN', 'QUIP', 
         'SWAG', 'VISA', 'WISH']

n_words = len(words)

coded_words = []
for wrd in words:
    num_word = []
    for let in wrd:
        num_word.append(let_to_num_dict[let])
    coded_words.append(num_word)
            
used_numbers = list(set([coded_words[wrd][let] for wrd in range(n_words) for let in range(4)]))
used_numbers.sort()

from ortools.constraint_solver import pywrapcp

solver = pywrapcp.Solver('four items')

x={nbr: solver.IntVar(0, 3, 'letter_{i}'.format(i=str(str(nbr)+num_to_let_dict[nbr])))
                      for nbr in used_numbers}  

## constraint that the allocations must differ across the words:
for wrd in coded_words:
    letters_on_word = [x[nbr] for nbr in wrd]
    solver.Add(solver.AllDifferent(letters_on_word))

x_flat = [x[nbr] for nbr in used_numbers]

db = solver.Phase(x_flat,
                    solver.CHOOSE_FIRST_UNBOUND,
                    solver.ASSIGN_MIN_VALUE)
  # Add variables to the solution collector.
solution = solver.Assignment()
solution.Add(x_flat)
collector = solver.AllSolutionCollector()
collector.Add(x_flat)
solutions_limit = solver.SolutionsLimit(1)
solver.Solve(db, [solutions_limit, collector])
print('sols', collector.SolutionCount())
for nbr in used_numbers:
    print('letter:', x[nbr], '  is in dice:', collector.Value(0, x[nbr]))

## puzzle 13

  Suppose we wish to schedule six one-hour lectures, v1, v2, v3, v4, v5, v6.
  Among the the potential audience there are people who wish to hear both
  
   - v1 and v2
   - v1 and v4
   - v3 and v5
   - v2 and v6
   - v4 and v5
   - v5 and v6
   - v1 and v6
  
  How many hours are necessary in order that the lectures can be given
  without clashes?
  
  https://github.com/google/or-tools/blob/master/examples/python/lectures.py

In [ ]:
from ortools.constraint_solver import pywrapcp
lectures = range(1,7)
conflicts = [[1, 2],
             [1, 4],
             [3, 5],
             [2, 6],
             [4, 5],
             [5, 6],
             [1, 6]]
from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("simple_CP", parameters)

# alocation variable
lecture_times = [solver.IntVar(0, len(lectures), 'lect_{lct}'.format(lct=lct))
                               for lct in lectures]

# constraints
for conf in conflicts:
    solver.Add(solver.AllDifferent([lecture_times[conf[0]-1],
                                    lecture_times[conf[1]-1]]))

# I want the solution that minimizes the max lecture time
obj_value = solver.Max(lecture_times)
objective = solver.Minimize(obj_value, 1)

# Create a solution collector.
collector = solver.LastSolutionCollector()
# Add the decision vardecision_builderiables.
collector.Add(lecture_times)
# Add the objective.
collector.AddObjective(obj_value)

db = solver.Phase(lecture_times,
                  solver.CHOOSE_FIRST_UNBOUND,
                  solver.ASSIGN_MIN_VALUE)

solver.Solve(db, [objective, collector])
if collector.SolutionCount() > 0:
    best_solution = collector.SolutionCount() - 1
    print("Objective value:", collector.ObjectiveValue(best_solution))
    print()
    for lct in lectures:
        print('lecture', lct, 'starts at:', collector.Value(best_solution, lecture_times[lct-1]))




#objective = solver.Maximize(obj_expr, 1)

In [ ]:
def main():

  # Create the solver.
  solver = pywrapcp.Solver('Lectures')

  #
  # data
  #

  #
  # The schedule requirements:
  # lecture a cannot be held at the same time as b
  # Note: 1-based
  g = [
      [1, 2],
      [1, 4],
      [3, 5],
      [2, 6],
      [4, 5],
      [5, 6],
      [1, 6]
  ]

  # number of nodes
  n = 6

  # number of edges
  edges = len(g)

  #
  # declare variables
  #
  v = [solver.IntVar(0, n - 1, 'v[%i]' % i) for i in range(n)]

  # maximum color, to minimize
  # Note: since Python is 0-based, the
  # number of colors is +1
  max_c = solver.IntVar(0, n - 1, 'max_c')

  #
  # constraints
  #
  solver.Add(max_c == solver.Max(v))

  # ensure that there are no clashes
  # also, adjust to 0-base
  for i in range(edges):
    solver.Add(v[g[i][0] - 1] != v[g[i][1] - 1])

  # symmetry breaking:
  # - v0 has the color 0,
  # - v1 has either color 0 or 1
  solver.Add(v[0] == 0)
  solver.Add(v[1] <= 1)

  # objective
  objective = solver.Minimize(max_c, 1)

  #
  # solution and search
  #
  db = solver.Phase(v,
                    solver.CHOOSE_MIN_SIZE_LOWEST_MIN,
                    solver.ASSIGN_CENTER_VALUE)

  solver.NewSearch(db, [objective])

  num_solutions = 0
  while solver.NextSolution():
    num_solutions += 1
    print('max_c:', max_c.Value() + 1, 'colors')
    print('v:', [v[i].Value() for i in range(n)])
    print()

  print('num_solutions:', num_solutions)
  print('failures:', solver.Failures())
  print('branches:', solver.Branches())
  print('WallTime:', solver.WallTime(), 'ms')


if __name__ == '__main__':
  main()

## puzzle - n queens 

https://github.com/google/or-tools/blob/master/examples/python/nqueens3.py

In [ ]:
from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("simple_CP", parameters)

def queen_solver(n_queens=8):
    
    queens = [solver.IntVar(0, n_queens-1, 'q_{q}'.format(q=q))
              for q in range(n_queens)]
    
    # every queen on differnt row
    solver.Add(solver.AllDifferent(queens))

    # no diagonals
    for q in range(n_queens-1):
        for i in range(1, n_queens-q):
            solver.Add(queens[q]!=queens[q+i]+i)
            solver.Add(queens[q]!=queens[q+i]-i)
            
    db = solver.Phase(queens,
                      solver.CHOOSE_FIRST_UNBOUND,
                      solver.ASSIGN_MIN_VALUE)
    # Add variables to the solution collector.
    solution = solver.Assignment()
    solution.Add(queens)
    collector = solver.AllSolutionCollector()
    collector.Add(queens)
    solutions_limit = solver.SolutionsLimit(1)
    solver.Solve(db, [solutions_limit, collector])
    print('sols', collector.SolutionCount())
    
    
    for row in range(n_queens):
        queen_position = collector.Value(n=0, var=queens[row])
        row_print = []
        for col in range(n_queens):
            if col == queen_position:
                row_print.append('*')
            else:
                row_print.append(':')
        print(row_print)

queen_solver(8)    

## puzzle - nurses

https://github.com/google/or-tools/blob/master/examples/python/nurses_cp.py

a hospital supervisor needs to create a weekly schedule for 4 nurses, subject to the following conditions:

Each day is divided into three 8-hour shifts.

On each day, all nurses are assigned to different shifts and one nurse has the day off.

Each nurse works five or six days a week.

No shift is staffed by more than two different nurses in a week.

If a nurse works shifts 2 or 3 on a given day, he must also work the same shift either the previous day or the following day.

In [ ]:
##### UNFINISHED! #############

from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("nurses", parameters)

n_nurses = 4
n_days = 7
n_shifts = 4 # shift 0 corresponds to being off

# nurses(day, shift)->returns which nurse is allocated to it
nurses = {}
for day in range(n_days):
    for shft in range(n_shifts):
        nurses[(day, shft)] = solver.IntVar(0, n_nurses,
                                             'nurse_day{day}_shift{shft}'.format(day=day, shft=shft))
        
        
## across the day, each shift must be staffed by a different nurse:
for day in range(n_days):
    nurses_of_the_day = [nurses[(day, shft)] for shft in range(n_shifts)]
    solver.Add(solver.AllDifferent(nurses_of_the_day))
    

# Each nurse works five or six days a week.
for nrs in range(n_nurses):
    solver.Add(solver.Sum([solver.Max([nurses[(day, shft)] == nrs
                                   for shft in range(1, n_shifts)]) == 1
                       for day in range(n_days)]) >=5)
    
    solver.Add(solver.Sum([solver.Max([nurses[(day, shft)] == nrs
                                   for shft in range(1, n_shifts)]) == 1
                       for day in range(n_days)]) <=6)


# No shift is staffed by more than two different nurses in a week.

## how many nurses staff  shift=sfht?
n = [nurses[(day, shft)] for day in range(n_days)]
 
    

In [ ]:
n1 = n[1]
n1.IndexOf(n==1)

In [ ]:
### Litlte toydiff ###

from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("nurses", parameters)

n_vars = 4
n_values = n_vars

variables = [solver.IntVar(0, n_values-1, 'x{var}'.format(var=var)) for var in range(n_vars)]

#solver.Add(solver.AllDifferent(variables))
solver.Add(solver.Count(variables, 0)==1)

obj_value = solver.Sum(variables)
objective = solver.Minimize(obj_value, 1)
# Create a solution collector.
collector = solver.LastSolutionCollector()
# Add the decision vardecision_builderiables.
collector.Add(variables)
# Add the objective.
collector.AddObjective(obj_value)

db = solver.Phase(variables,
                  solver.CHOOSE_FIRST_UNBOUND,
                  solver.ASSIGN_MIN_VALUE)


solver.Solve(db, [objective, collector])

if collector.SolutionCount() > 0:
    best_solution = collector.SolutionCount() - 1
    print("Objective value:", collector.ObjectiveValue(best_solution))
    print()
    for var in range(n_vars):
        print('var', var, 'is:', collector.Value(best_solution, variables[var]))





In [ ]:
### THE SOLUTION VERSION ###
from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("nurses", parameters)

n_nurses = 4
n_days = 7
n_shifts = 4 # shift 0 corresponds to being off

# nurses(day, shift)->returns which nurse is allocated to it
nurses = {}
for day in range(n_days):
    for shft in range(n_shifts):
        nurses[(day, shft)] = solver.IntVar(0, n_nurses,
                                             'nurse_day{day}_shift{shft}'.format(day=day, shft=shft))
        
        
## across the day, each shift must be staffed by a different nurse:
for day in range(n_days):
    nurses_of_the_day = [nurses[(day, shft)] for shft in range(n_shifts)]
    solver.Add(solver.AllDifferent(nurses_of_the_day))
    

# Each nurse works five or six days a week.
for nrs in range(n_nurses):
    solver.Add(solver.Sum([solver.Max([nurses[(day, shft)] == nrs
                                   for shft in range(1, n_shifts)]) == 1
                       for day in range(n_days)]) >=5)
    
    solver.Add(solver.Sum([solver.Max([nurses[(day, shft)] == nrs
                                   for shft in range(1, n_shifts)]) == 1
                       for day in range(n_days)]) <=6)
    

# No shift is staffed by more than two different nurses in a week.   
# shifts(day, nrs) -> returns which shift is being done by nurse nrs in that day

shifts = {}
for day in range(n_days):
    for nrs in range(n_nurses):
        shifts[(day,nrs)] = solver.IntVar(0, n_shifts,
                                          'shift_nrs{nrs}_day{day}'.format(nrs=nrs, day=day))

# solver.Add(x.IndexOf(y) == j)
# adds the constraint that y[x] == j.
for day in range(n_days):
    shifts_for_day_nurses = [shifts[(day, nrs)] for nrs in range(n_nurses)]
    nurses_for_day_shifts = [nurses[(day, shft)] for shft in range(n_shifts)]
    
    # [shft_of_n0, shft_of_n1, shft_of_n2, shft_of_n3]
    # [nrs_of_sh0, nrs_of_sh1, nrs_of_sh2, nrs_of_sh3]
    
    for shft in range(n_shifts):
        n = nurses_for_day_shifts[shft]
        solver.Add(n.IndexOf(shifts_for_day_nurses) == shft)

    
# No shift is staffed by more than two different nurses in a week.
for 


## puzzle - organize day
###### https://github.com/google/or-tools/blob/master/examples/python/organize_day.py

In [ ]:
#data 
n = 4
tasks = list(range(n))
work, mail, shop, bank = tasks
durations = [4, 1, 2, 1]

before_tasks = [
      [bank, shop],
      [mail, work]
]

begin_time = 9
end_time = 17

### 
from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("nurses", parameters)

start_times = [solver.IntVar(begin_time, end_time, 'task{tsk}'.format(tsk=tsk)) 
               for tsk in tasks]

## all start times must be after begin time 
## and all start+durations must be before end time
for tsk in tasks:
    solver.Add(start_times[tsk]>=begin_time)
    solver.Add(start_times[tsk]+durations[tsk]<=end_time)


# no overlap
for tsk in tasks:
    for next_tsk in tasks:
        if tsk!=next_tsk:
            # next task cannot start inside [start, start+duration]
            #solver.Add(
            #    solver.Sum([start_times[next_tsk]<start_times[tsk]+durations[tsk],
            #                start_times[next_tsk]>start_times[tsk]])<2)
            b1=solver.IsLessVar(start_times[next_tsk],start_times[tsk]+durations[tsk])
            b2=solver.IsLessVar(start_times[tsk],start_times[next_tsk])
            solver.Add(b1 + b2 < 2)               

            # next task cannot end inside [start, start+duration]
            solver.Add(
                solver.Sum([durations[next_tsk]+start_times[next_tsk]<start_times[tsk]+durations[tsk],
                            durations[next_tsk]+start_times[next_tsk]>start_times[tsk]])<2)

# the required order
for con in before_tasks:
    solver.Add(start_times[con[0]]<start_times[con[1]])
    

collector = solver.LastSolutionCollector()
# Add the decision vardecision_builderiables.
collector.Add(start_times)
# Add the objective.

db = solver.Phase(start_times,
                  solver.CHOOSE_FIRST_UNBOUND,
                  solver.ASSIGN_MIN_VALUE)


solver.Solve(db, [collector])

if collector.SolutionCount() > 0:
    best_solution = collector.SolutionCount() - 1
    print("Objective value:", collector.ObjectiveValue(best_solution))
    print()
    for var in range(n):
        print('task', var, 'at:', collector.Value(best_solution, start_times[var]))



## Puzzle - P-median
##### https://github.com/google/or-tools/blob/master/examples/python/p_median.py

In [ ]:
demand = [100, 80, 80, 70]

## distance [doemand node][distribution node]
distance = [
      [2, 10, 50],
      [2, 10, 52],
      [50, 60, 3],
      [40, 60, 1]
  ]

clients = range(len(demand))
warehouses = range(len(distance[0]))
max_open = 2

from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("nurses", parameters)

ships_to = {}
for whr in warehouses:
    for clt in clients:
        ships_to[(whr, clt)] = solver.IntVar(0, 1,
                                              'whr{whr}_clt{clt}'.format(whr=whr, clt=clt))


# only max_open distribution nodes are allowed open:
solver.Add(solver.Sum([solver.Sum([ships_to[(whr, clt)] 
                                   for clt in clients])>0
                       for whr in warehouses])==max_open)


# every client must be served by a warehouse
for clt in clients:
    solver.Add(solver.Sum([ships_to[(whr, clt)] for whr in warehouses])==1)



## minimize the weighted distance
# demand weighted distnace from wherehouse to client.
# boolean if wherehouse is distributing to vlient
obj_value = solver.Sum([distance[clt][whr]*demand[clt]*ships_to[(whr,clt)]
                        for whr in warehouses for clt in clients])

objective = solver.Minimize(obj_value, 1)


# solution collector
collector = solver.LastSolutionCollector()

# Add the decision variables.
x = [ships_to[(whr,clt)]
     for whr in warehouses for clt in clients]
collector.Add(x)

# Add the objective.
collector.AddObjective(obj_value)

db = solver.Phase(x,
                  solver.INT_VAR_DEFAULT,
                  solver.INT_VALUE_DEFAULT)


solver.Solve(db, [objective, collector])

print('WallTime:', solver.WallTime(), 'ms')


if collector.SolutionCount() > 0:
    best_solution = collector.SolutionCount() - 1
    print("Objective value:", collector.ObjectiveValue(best_solution))
    print()
    for whr in warehouses:
         for clt in clients:
             print('whr=', whr, 'clt=', clt, 'is:',
                   collector.Value(best_solution, ships_to[(whr, clt)]))





## puzzle - sitting people in their sports

  Betty, Chris, Donald, Fred, Gary, Mary, and Paul want to align in one
  row for taking a photo. Some of them have preferences next to whom
  they want to stand:
     1. Betty wants to stand next to Gary and Mary.
     2. Chris wants to stand next to Betty and Gary.
     3. Fred wants to stand next to Mary and Donald.
     4. Paul wants to stand next to Fred and Donald.
  Obviously, it is impossible to satisfy all preferences. Can you find
  an alignment that maximizes the number of satisfied preferences?


 https://github.com/google/or-tools/blob/master/examples/python/photo_problem.py

In [ ]:
from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("nurses", parameters)


people = range(7)
#b=0  // #c=1  //  #d=2  //  #f=3  //  #g=4  //  #m=5  //  #p=6
#######
# 0 wants to be close to 4
# 0 wants to be close to 5
# 1 wants to be close to 0
# 1 wants to be close to 4
# 3 wants to be close to 5
# 3 wants to be close to 2
# 6 wants to be close to 3
# 6 wants to be close to 2

seatings = [solver.IntVar(0, len(people)-1, 'person{p}'.format(p=p)) for p in people]

## every seating is different ##
solver.Add(solver.AllDifferent(seatings))


prefs = solver.IntVar(0, 8, 'prefs')
p1 = solver.IsEqualCstVar(abs(seatings[0]-seatings[4]), 1)
p2 = solver.IsEqualCstVar(abs(seatings[0]-seatings[5]), 1)
p3 = solver.IsEqualCstVar(abs(seatings[1]-seatings[0]), 1)
p4 = solver.IsEqualCstVar(abs(seatings[1]-seatings[4]), 1)
p5 = solver.IsEqualCstVar(abs(seatings[3]-seatings[5]), 1)
p6 = solver.IsEqualCstVar(abs(seatings[3]-seatings[2]), 1)
p7 = solver.IsEqualCstVar(abs(seatings[6]-seatings[3]), 1)
p8 = solver.IsEqualCstVar(abs(seatings[6]-seatings[2]), 1)
p_list = [p1, p2, p3, p4, p5, p6, p7, p8]

solver.Add(prefs == solver.Sum(p_list))



objective = solver.Maximize(prefs, 1)


# solution collector
collector = solver.LastSolutionCollector()

# Add the decision variables.
collector.Add(seatings)

# Add the objective.
collector.AddObjective(prefs)
collector.Add(p_list)

db = solver.Phase(seatings+[prefs],
                  solver.INT_VAR_DEFAULT,
                  solver.INT_VALUE_DEFAULT)


solver.Solve(db, [objective, collector])

print('WallTime:', solver.WallTime(), 'ms')


if collector.SolutionCount() > 0:
    best_solution = collector.SolutionCount() - 1
    print("Objective value:", collector.ObjectiveValue(best_solution))
    print()
    for p in people:
        print(p, 'seats at', collector.Value(best_solution, seatings[p]))
        
    for prf in range(len(p_list)):
        print('pref', prf, 'is satisfied or not:',
              collector.Value(best_solution, p_list[prf]))




## post office 
https://github.com/google/or-tools/blob/master/examples/python/post_office_problem2.py

  A post office requires a different number of full-time employees working
  on different days of the week [summarized below]. Union rules state that
  each full-time employee must work for 5 consecutive days and then receive
  two days off. For example, an employee who works on Monday to Friday
  must be off on Saturday and Sunday. The post office wants to meet its
  daily requirements using only full-time employees. Minimize the number
  of employees that must be hired.
  To summarize the important information about the problem:
    * Every full-time worker works for 5 consecutive days and takes 2 days off
    * Day 1 (Monday): 17 workers needed
    * Day 2 : 13 workers needed
    * Day 3 : 15 workers needed
    * Day 4 : 19 workers needed
    * Day 5 : 14 workers needed
    * Day 6 : 16 workers needed
    * Day 7 (Sunday) : 11 workers needed
  The post office needs to minimize the number of employees it needs
  to hire to meet its demand.




In [ ]:
from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("post", parameters)

workers = range(20)
days = range(0,7)
day_staff_needs = [17, 13, 15, 19, 14, 16, 11]


#hires = [solver.IntVar(0,1,'wrk{wrk}'.format(wrk=wrk)) for wrk in workers]

#staffing[(day, wrk)] =1/0 (is working or not)
staffing = {}
for day in days:
    for wrk in workers:
        staffing[(day, wrk)] = solver.IntVar(0,1,
                                             'wrk{wrk}_day{day}'.format(wrk=wrk, day=day))


### satisfy the dailly staffing needs
for day in days:
    workers_in_day = solver.Sum([staffing[(day, wrk)] for wrk in workers])
    solver.AddConstraint(workers_in_day >= day_staff_needs[day])
    
### maximize 0s in workers
#zeroed_workers = solver.IntVar(0, 30)
#worker_in_day_binary =[]
#for wrk in workers:
#    worker_in_day_binary.append(
#        solver.IsEqualCstVar(solver.Sum([staffing[(day, wrk)] for day in days]), 0))
#    
#solver.Add(zeroed_workers == solver.Sum(worker_in_day_binary))
#objective = solver.Maximize(zeroed_workers, 1)

worker_in_day_binaries =[]
for wrk in workers:
    worker_in_day_binaries.append(solver.Max([staffing[(day, wrk)] for day in days]))

obj_value = solver.Sum(worker_in_day_binaries)
objective = solver.Minimize(obj_value, 1)


# solution collector
collector = solver.LastSolutionCollector()


# Add the decision variables.
staffing_flat = [staffing[(day,wrk)] for day in days for wrk in workers]
collector.Add(staffing_flat)

# Add the objective.
collector.AddObjective(obj_value)
collector.Add(worker_in_day_binaries)

db = solver.Phase(staffing_flat + worker_in_day_binaries,
                  solver.INT_VAR_DEFAULT,
                  solver.INT_VALUE_DEFAULT)


solver.Solve(db, [objective, collector])

print('WallTime:', solver.WallTime(), 'ms')


if collector.SolutionCount() > 0:
    best_solution = collector.SolutionCount() - 1
    print("Objective value:", collector.ObjectiveValue(best_solution))
    print()
    for wrk in workers:
        rota = [collector.Value(best_solution, staffing[(day, wrk)]) for day in days]
        print(rota)
        
    print('')
    for i in staffing_flat:
        print(collector.Value(best_solution, i))
        

## puzzle - some set coverage 
https://github.com/google/or-tools/blob/master/examples/python/set_covering3.py
    
10 senators making a committee, where there must at least be one
  representative from each group:
 * group:        senators:
 * southern      1 2 3 4 5
 * northern      6 7 8 9 10
 * liberals      2 3 8 9 10
 * conservative  1 5 6 7
 * democrats     3 4 5 6 7 9
 * republicans   1 2 8 10
 
The objective is to minimize the number of senators.


In [14]:
from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("post", parameters)

southern = [1, 2, 3, 4, 5]
northern = [6, 7, 8, 9, 10]
liberals =  [2, 3, 8, 9,10]
conservative = [1, 5, 6, 7]
democrats = [3, 4, 5, 6, 7, 9]
republicans = [1, 2, 8, 10]

sets = [southern, northern, liberals, conservative, democrats, republicans]


x = [solver.IntVar(0,1, 'sen{s}'.format(s=s+1)) for s in range(0,10)]

# set coverage
for st in sets:
    solver.Add(solver.Sum([x[sen-1] for sen in st])>0)

obj_value = solver.Sum(x)
objective = solver.Minimize(obj_value, 1)

# solution collector
collector = solver.LastSolutionCollector()


# Add the decision variables.
collector.Add(x)

# Add the objective.
collector.AddObjective(obj_value)

db = solver.Phase(x,
                  solver.INT_VAR_DEFAULT,
                  solver.INT_VALUE_DEFAULT)


solver.Solve(db, [objective, collector])

print('WallTime:', solver.WallTime(), 'ms')
if collector.SolutionCount() > 0:
    best_solution = collector.SolutionCount() - 1
    print("Objective value:", collector.ObjectiveValue(best_solution))
    print()
    for i in x:
        print(i, collector.Value(best_solution, i))
    


WallTime: 1 ms
Objective value: 2

sen1(0 .. 1) 0
sen2(0 .. 1) 0
sen3(0 .. 1) 0
sen4(0 .. 1) 0
sen5(0 .. 1) 1
sen6(0 .. 1) 0
sen7(0 .. 1) 0
sen8(0 .. 1) 0
sen9(0 .. 1) 0
sen10(0 .. 1) 1


## puzzle - set deployment

https://github.com/google/or-tools/blob/master/examples/python/set_covering_deployment.py

In [4]:

from ortools.constraint_solver import pywrapcp
parameters = pywrapcp.Solver.DefaultSolverParameters()
solver = pywrapcp.Solver("post", parameters)

countries = ["Alexandria",
               "Asia Minor",
               "Britain",
               "Byzantium",
               "Gaul",
               "Iberia",
               "Rome",
               "Tunis"]
n_provs = len(countries)

  # the incidence matrix (neighbours)
mat = [
      [0, 1, 0, 1, 0, 0, 1, 1],
      [1, 0, 0, 1, 0, 0, 0, 0],
      [0, 0, 0, 0, 1, 1, 0, 0],
      [1, 1, 0, 0, 0, 0, 1, 0],
      [0, 0, 1, 0, 0, 1, 1, 0],
      [0, 0, 1, 0, 1, 0, 1, 1],
      [1, 0, 0, 1, 1, 1, 0, 1],
      [1, 0, 0, 0, 0, 1, 1, 0]]

# does province get a first division
x = [solver.IntVar(0, 1, 'x{prov}'.format(prov=prov)) for prov in range(n_provs)]

# province get a second division 
y = [solver.IntVar(0, 1, 'y{prov}'.format(prov=prov)) for prov in range(n_provs)]

### each province is either x[i]=1 or is neighbor to a y[i]=1
for prov in range(n_provs):
    tmp1 = [x[prov]]
    tmp2 = [y[i] for i in range(n_provs) if mat[prov][i]==1]
    tmp = tmp1 + tmp2
    m = solver.Max(tmp)
    solver.Add(m > 0)
    
## you can only get a second division (y) if you get a first (x)

for prov in range(n_provs):
    solver.Add(x[prov]>=y[prov])
    
obj_value = solver.Sum(x) + solver.Sum(y)
objective = solver.Minimize(obj_value, 1)

# solution collector
collector = solver.LastSolutionCollector()

# Add the decision variables.
collector.Add(x)
collector.Add(y)

# Add the objective.
collector.AddObjective(obj_value)

db = solver.Phase(x+y,
                  solver.INT_VAR_DEFAULT,
                  solver.INT_VALUE_DEFAULT)


solver.Solve(db, [objective, collector])

print('WallTime:', solver.WallTime(), 'ms')
if collector.SolutionCount() > 0:
    best_solution = collector.SolutionCount() - 1
    print("Objective value:", collector.ObjectiveValue(best_solution))
    print()
    for i in x:
        print(i, collector.Value(best_solution, i))
    for i in y:
        print(i, collector.Value(best_solution, i))
    


WallTime: 1 ms
Objective value: 4

x0(0 .. 1) 0
x1(0 .. 1) 0
x2(0 .. 1) 0
x3(0 .. 1) 1
x4(0 .. 1) 0
x5(0 .. 1) 1
x6(0 .. 1) 0
x7(0 .. 1) 0
y0(0 .. 1) 0
y1(0 .. 1) 0
y2(0 .. 1) 0
y3(0 .. 1) 1
y4(0 .. 1) 0
y5(0 .. 1) 1
y6(0 .. 1) 0
y7(0 .. 1) 0


## puzzle - ski
https://github.com/google/or-tools/blob/master/examples/python/ski_assignment.py

In [18]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver('CoinsGridCBC',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

n_items = 6
n_people = 5
ski_heights = [1, 2, 5, 7, 13, 21]
skier_heights = [3, 4, 7, 11, 18]

allocation = {}
for itm in range(n_skis):
    for p in range(n_skiers):
        allocation[(itm, p)]= solver.IntVar(0, 1, 'a_itm{itm}_skier{p}'.format(itm=itm, p=p))
    
# one item per person
for p in range(n_skiers):
    items_per_person = solver.Sum([allocation[(itm, p)] for itm in range(n_skis)])
    solver.Add(items_per_person==1)

# obj value
obj_value = solver.Sum([allocation[(itm, p)]*abs(ski_heights[itm] - skier_heights[p])
                       for itm in range(n_items) for p in range(n_skiers)])

objective = solver.Minimize(obj_value)


result_status = solver.Solve()
  # The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

  # The solution looks legit (when using solvers other than
  # GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended!).
assert solver.VerifySolution(1e-7, True)

print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())

# The objective value of the solution.
print('Optimal objective value = %d' % solver.Objective().Value())
print()
for itm in range(n_skis):
    for p in range(n_skiers):
        variable  = allocation[(itm, p)]
        print('%s = %d' % (variable.name(), variable.solution_value()))

Number of variables = 30
Number of constraints = 5
Optimal objective value = 7

a_itm0_skier0 = 0
a_itm0_skier1 = 0
a_itm0_skier2 = 0
a_itm0_skier3 = 0
a_itm0_skier4 = 0
a_itm1_skier0 = 1
a_itm1_skier1 = 0
a_itm1_skier2 = 0
a_itm1_skier3 = 0
a_itm1_skier4 = 0
a_itm2_skier0 = 0
a_itm2_skier1 = 1
a_itm2_skier2 = 0
a_itm2_skier3 = 0
a_itm2_skier4 = 0
a_itm3_skier0 = 0
a_itm3_skier1 = 0
a_itm3_skier2 = 1
a_itm3_skier3 = 0
a_itm3_skier4 = 0
a_itm4_skier0 = 0
a_itm4_skier1 = 0
a_itm4_skier2 = 0
a_itm4_skier3 = 1
a_itm4_skier4 = 0
a_itm5_skier0 = 0
a_itm5_skier1 = 0
a_itm5_skier2 = 0
a_itm5_skier3 = 0
a_itm5_skier4 = 1


In [19]:
from ortools.constraint_solver import pywrapcp


# pylint: disable=too-many-statements
def main():
  # Create the solver.
  solver = pywrapcp.Solver('zebra')

  red = solver.IntVar(1, 5, 'red')
  green = solver.IntVar(1, 5, 'green')
  yellow = solver.IntVar(1, 5, 'yellow')
  blue = solver.IntVar(1, 5, 'blue')
  ivory = solver.IntVar(1, 5, 'ivory')

  englishman = solver.IntVar(1, 5, 'englishman')
  spaniard = solver.IntVar(1, 5, 'spaniard')
  japanese = solver.IntVar(1, 5, 'japanese')
  ukrainian = solver.IntVar(1, 5, 'ukrainian')
  norwegian = solver.IntVar(1, 5, 'norwegian')

  dog = solver.IntVar(1, 5, 'dog')
  snails = solver.IntVar(1, 5, 'snails')
  fox = solver.IntVar(1, 5, 'fox')
  zebra = solver.IntVar(1, 5, 'zebra')
  horse = solver.IntVar(1, 5, 'horse')

  tea = solver.IntVar(1, 5, 'tea')
  coffee = solver.IntVar(1, 5, 'coffee')
  water = solver.IntVar(1, 5, 'water')
  milk = solver.IntVar(1, 5, 'milk')
  fruit_juice = solver.IntVar(1, 5, 'fruit juice')

  old_gold = solver.IntVar(1, 5, 'old gold')
  kools = solver.IntVar(1, 5, 'kools')
  chesterfields = solver.IntVar(1, 5, 'chesterfields')
  lucky_strike = solver.IntVar(1, 5, 'lucky strike')
  parliaments = solver.IntVar(1, 5, 'parliaments')

  solver.Add(solver.AllDifferent([red, green, yellow, blue, ivory]))
  solver.Add(solver.AllDifferent([englishman, spaniard, japanese,
                                  ukrainian, norwegian]))
  solver.Add(solver.AllDifferent([dog, snails, fox, zebra, horse]))
  solver.Add(solver.AllDifferent([tea, coffee, water, milk,
                                  fruit_juice]))
  solver.Add(solver.AllDifferent([parliaments, kools, chesterfields,
                                  lucky_strike, old_gold]))

  solver.Add(englishman == red)
  solver.Add(spaniard == dog)
  solver.Add(coffee == green)
  solver.Add(ukrainian == tea)
  solver.Add(green == ivory + 1)
  solver.Add(old_gold == snails)
  solver.Add(kools == yellow)
  solver.Add(milk == 3)
  solver.Add(norwegian == 1)
  solver.Add(abs(fox - chesterfields) == 1)
  solver.Add(abs(horse - kools) == 1)
  solver.Add(lucky_strike == fruit_juice)
  solver.Add(japanese == parliaments)
  solver.Add(abs(norwegian - blue) == 1)

  all_vars = [parliaments, kools, chesterfields, lucky_strike, old_gold,
              englishman, spaniard, japanese, ukrainian, norwegian,
              dog, snails, fox, zebra, horse,
              tea, coffee, water, milk, fruit_juice,
              red, green, yellow, blue, ivory]

  solver.NewSearch(solver.Phase(all_vars,
                                solver.INT_VAR_DEFAULT,
                                solver.INT_VALUE_DEFAULT))
  if solver.NextSolution():
    people = [englishman, spaniard, japanese, ukrainian, norwegian]
    water_drinker = [p for p in people if p.Value() == water.Value()][0]
    zebra_owner = [p for p in people if p.Value() == zebra.Value()][0]
    print('The', water_drinker.Name(), 'drinks water.')
    print('The', zebra_owner.Name(), 'owns the zebra.')
  else:
    print('No solutions to the zebra problem, this is unusual!')
  solver.EndSearch()


if __name__ == '__main__':
  main()

The norwegian drinks water.
The japanese owns the zebra.
